## Config

In [1]:
# parameters
SAVE_OUTPUT = True
SIZE_PLOTS = (12,10)
BUFFER_SIZE = 402.336 
YEARS = [2017,2018,2019,2020,2021,2022,2023]

#Location of the data
INPUT_RAW_DATA_PATH = "../data/raw/"
INPUT_INTERIM_DATA_PATH = "../data/interim/"
INPUT_PROCESSED_DATA_PATH = "../data/processed/"
OUTPUT_DATA_PATH = "../data/interim/"

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geoarrow.pandas as _
import plotly.express as px
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


## Load data

In [3]:
cunters_list = []
#for year in years, load the data in counters_data{year}.parquet
for year in YEARS:
    #Load the data
    counters_data = pd.read_parquet(f"{INPUT_INTERIM_DATA_PATH}counters_data{year}.parquet")
    counters_data = counters_data.rename(columns={'id':'Id_aforament'})
    counters_data["year"]=year
    #Load the geodata
    stations_data = gpd.read_parquet(f"{INPUT_INTERIM_DATA_PATH}stations_data{year}.parquet")

    # Ensure the 'Id_aforament' column is of the same type in both dataframes
    counters_data['Id_aforament'] = counters_data['Id_aforament'].astype(int)
    stations_data['Id_aforament'] = stations_data['Id_aforament'].astype(int)
    counters_data = pd.merge(counters_data, stations_data, on='Id_aforament')

    #Append the data to the list
    cunters_list.append(counters_data)
    
counters_data = pd.concat(cunters_list)
counters_data = gpd.GeoDataFrame(counters_data, geometry='geometry')




## Data management

In [4]:
print(counters_data.shape)
print(counters_data.columns)
counters_data.head()

(54037979, 15)
Index(['Id_aforament', 'date', 'weekday', 'hour', 'intensity', 'error', 'year',
       'Desc_aforament', 'Num_carrils', 'Codi_districte', 'Codi_barri',
       'Codi_tipus_equip_mesura', 'Desc_tipus_equip_mesura', 'geometry',
       'Codi_Barri'],
      dtype='object')


,Id_aforament,date,weekday,hour,intensity,error,year,Desc_aforament,Num_carrils,Codi_districte,Codi_barri,Codi_tipus_equip_mesura,Desc_tipus_equip_mesura,geometry,Codi_Barri
0,20001,2017-01-01,Sunday,00:00:00.0000000,0,100.0,2017,DIPUTACIÓ - COMTE D'URGELL (carril BICI),1,2.0,8.0,1.0,Espira,POINT (2.15709 41.38300),NaN
1,20001,2017-01-01,Sunday,00:15:00.0000000,0,100.0,2017,DIPUTACIÓ - COMTE D'URGELL (carril BICI),1,2.0,8.0,1.0,Espira,POINT (2.15709 41.38300),NaN
2,20001,2017-01-01,Sunday,00:30:00.0000000,0,100.0,2017,DIPUTACIÓ - COMTE D'URGELL (carril BICI),1,2.0,8.0,1.0,Espira,POINT (2.15709 41.38300),NaN
3,20001,2017-01-01,Sunday,00:45:00.0000000,0,100.0,2017,DIPUTACIÓ - COMTE D'URGELL (carril BICI),1,2.0,8.0,1.0,Espira,POINT (2.15709 41.38300),NaN
4,20001,2017-01-01,Sunday,01:00:00.0000000,0,100.0,2017,DIPUTACIÓ - COMTE D'URGELL (carril BICI),1,2.0,8.0,1.0,Espira,POINT (2.15709 41.38300),NaN


In [5]:
counters_data["year"].value_counts()

year
2023    12370752
2022    10612512
2021     8916672
2020     8134848
2019     7393694
2018     4448766
2017     2160735
Name: count, dtype: int64

## Deal Errors

### Visualize

In [7]:
conditions = [
    (counters_data["error"] == 0.0),
    (counters_data["error"] == 100.0),
    (counters_data["error"].isna()),

    (counters_data["error"] != 0.0) & (counters_data["error"] != 100.0)
]

choices = ["NO ERROR", "ERROR", "NAN", "PARTIAL ERROR"]

counters_data["ERROR"] = np.select(conditions, choices, default="NO ERROR")

observations_per_day = counters_data.groupby(['date', 'ERROR'])['intensity'].count().reset_index(name='observations')




In [8]:
fig = px.area(observations_per_day, x='date', y=observations_per_day['observations'],
              color='ERROR', 
              title='Number of counts per day by error type', 
              labels={'date': 'Date', 'counts': 'Number of counts', 'ERROR': 'Error Type'},
              category_orders={'ERROR': ['NO ERROR','NAN', 'PARTIAL ERROR','ERROR' ]},
              color_discrete_map={'NO ERROR': 'green', 'PARTIAL ERROR': 'orange','NAN':"yellow", 'ERROR': 'red'})
              
fig.show()

### Drop Erroneous values

In [9]:
print(counters_data["ERROR"].value_counts(dropna=False))


ERROR
NO ERROR         48623146
ERROR             3296714
NAN               1363399
PARTIAL ERROR      754720
Name: count, dtype: int64


In [10]:
counters_data.loc[counters_data['error'] != 0.0, "intensity"] = np.nan



In [11]:
print(counters_data["intensity"].isna().sum())
print(counters_data["error"].value_counts(dropna=False))


5414833
error
0.0      48623146
100.0     3296714
NaN       1363399
33.0       375484
67.0       370989
50.0         6579
25.0         1499
75.0          138
60.0           17
80.0           14
Name: count, dtype: int64


In [12]:
print(counters_data.columns)
print(counters_data.dtypes)
print(counters_data.shape)

Index(['Id_aforament', 'date', 'weekday', 'hour', 'intensity', 'error', 'year',
       'Desc_aforament', 'Num_carrils', 'Codi_districte', 'Codi_barri',
       'Codi_tipus_equip_mesura', 'Desc_tipus_equip_mesura', 'geometry',
       'Codi_Barri', 'ERROR'],
      dtype='object')
Id_aforament                        int32
date                       datetime64[ns]
weekday                            object
hour                               object
intensity                         float64
error                             float64
year                                int64
Desc_aforament                     object
Num_carrils                         int64
Codi_districte                    float64
Codi_barri                        float64
Codi_tipus_equip_mesura           float64
Desc_tipus_equip_mesura            object
geometry                         geometry
Codi_Barri                        float64
ERROR                              object
dtype: object
(54037979, 16)


## MICE

In [53]:
df_train = counters_data.loc[:, ['Id_aforament', 'date',  'hour', 'intensity', 'geometry']].sample(n=10000, random_state=42)

df_train['longitude'] = df_train.geometry.x
df_train['latitude'] = df_train.geometry.y
df_train=df_train.drop(columns=["geometry"])

df_train["date"] = pd.to_datetime(df_train["date"])
df_train["date"] = df_train["date"].astype('int64') // 10**9

df_train["quarter"] = df_train["hour"].str[3:5].astype(int)

df_train["hour"] = df_train["hour"].str[:2].astype(int)

print(df_train.shape)
print(df_train['intensity'].isna().sum())
print(df_train["intensity"].describe())
df_train.head()

(10000, 7)
987
count    9013.000000
mean       38.932320
std        90.876281
min         0.000000
25%         4.000000
50%        19.000000
75%        50.000000
max      3895.000000
Name: intensity, dtype: float64


,Id_aforament,date,hour,intensity,longitude,latitude,quarter
9081456,20274,1649116800,12,NaN,2.182220,41.405180,0
5379781,20174,1578009600,9,21.0,2.185905,41.416072,15
7652014,20233,1655596800,11,8.0,2.184223,41.430133,30
2954706,20117,1541116800,2,0.0,2.147358,41.390867,0
11903301,20384,1694908800,17,40.0,2.191306,41.418822,15


In [54]:
imputer = IterativeImputer(random_state=100, max_iter=10)
# fit on the dataset
imputer.fit(df_train)


IterativeImputer(random_state=100)

In [55]:
df_imputed = imputer.transform(df_train)

# Fill the missing values with the imputed values
df_train['intensity'] = df_imputed[:, 3]

# Ensure that the imputed values are not negative
df_train['intensity'] = df_train['intensity'].clip(lower=0)

# Check for any remaining NaN values and describe the column
print(df_train['intensity'].isna().sum())
df_train['intensity'].describe()

0


count    10000.000000
mean        38.922353
std         89.109085
min          0.000000
25%          5.000000
50%         20.000000
75%         48.925320
max       3895.000000
Name: intensity, dtype: float64

## Impute using MCMC

In [40]:
# Assuming 'gdf' is your original GeoDataFrame
gdf = counters_data[['date', 'hour', 'intensity', 'geometry']].sample(n=1000, random_state=0).copy()

# Extract spatial coordinates
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y

# Convert date and hour to datetime and time features
gdf['day_of_year'] = gdf['date'].dt.dayofyear
gdf['hour_of_day'] = pd.to_numeric(gdf['hour'], errors='coerce')

# Drop rows with NaN in 'hour_of_day' if they exist
gdf = gdf.dropna(subset=['hour_of_day'])

# Identify missing indices
missing_mask = gdf['intensity'].isna()

# Define the Bayesian model
with pm.Model() as model:
    # Priors for coefficients
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    beta_day = pm.Normal('beta_day', mu=0, sigma=10)
    beta_hour = pm.Normal('beta_hour', mu=0, sigma=10)
    beta_long = pm.Normal('beta_long', mu=0, sigma=10)
    beta_lat = pm.Normal('beta_lat', mu=0, sigma=10)
    
    # Predictors
    day_of_year = gdf['day_of_year'].values
    hour_of_day = gdf['hour_of_day'].values
    longitude = gdf['longitude'].values
    latitude = gdf['latitude'].values
    
    # Expected intensity value (linear combination of features)
    mu = (alpha + beta_day * day_of_year + beta_hour * hour_of_day +
          beta_long * longitude + beta_lat * latitude)
    
    # Likelihood for observed data
    sigma = pm.HalfNormal('sigma', sigma=10)
    observed_intensity = pm.Normal('observed_intensity', mu=mu[~missing_mask], sigma=sigma,
                                   observed=gdf.loc[~missing_mask, 'intensity'])
    
    # Impute missing intensities
    imputed_intensity = pm.Normal('imputed_intensity', mu=mu[missing_mask], sigma=sigma,
                                  shape=missing_mask.sum())
    
    # Posterior sampling
    trace = pm.sample(1000, tune=500, chains=4, random_seed=0)

NameError: name 'pm' is not defined

## Evaluate

## Save output

In [ ]:
if SAVE_OUTPUT:
    df.to_parquet(f'{OUTPUT_DATA_PATH}/file.parquet')

## Watermark

In [ ]:
!python -m pip install watermark --quiet

In [ ]:
%load_ext watermark

In [ ]:
%watermark

Last updated: 2024-08-23T15:55:33.641180+00:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.85+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [ ]:
%watermark --iversions

json  : 2.0.9
pandas: 2.1.4
google: 2.0.3
numpy : 1.26.4



In [ ]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy
